# Hackathon Query Notebook

Welcome to the hackathon! This notebook is your starting point for querying the main dataset.

It's designed to connect to a **remote Tentris server** that is already running and loaded with all the data. You do not need to run or install Tentris yourself.

**Your only task:** Follow the setup steps, run the cells, and start writing your own queries!

### Step 1: Setup env

Follow the instructions in the (README.md)[https://github.com/sdsc-ordes/open-pulse-quickstart/blob/feat/tentris-quickstart/src/tentris-quickstart/README.md]

### Step 2: Imports and Configuration

This cell imports the libraries and sets up our connection variables.

In [2]:
import rdflib
import pandas as pd
import tentris # Required to register the Tentris store
from tentris import TentrisHTTPStore
from IPython.display import display, Markdown

# --- 💡 IMPORTANT 💡 ---
# This is the ONLY line you need to change.
# Set this to the server IP address provided by the hackathon organizers.
ENDPOINT_URL = "http://128.178.219.51:7502"
# ------------------------

# We will create our main 'graph' object in the next step
graph = None

### Step 3: Connect to the Server

This cell creates the `rdflib.Graph` object. It uses the `TentrisHTTPStore`, which is optimized to work with the Tentris server.

**Note:** This cell also runs a test query (`ASK { ?s ?p ?o }`) to make sure the server is reachable. If this cell fails, please double-check the `ENDPOINT_URL` you set in Step 2.

In [3]:
display(Markdown(f"🚀 Connecting to server at `{ENDPOINT_URL}`..."))

try:
    # Initialize the TentrisHTTPStore with the base endpoint URL.
    # This client is smart and knows how to find the /sparql and /stream endpoints.
    store = TentrisHTTPStore(ENDPOINT_URL)
    
    # Create the graph object
    graph = rdflib.Graph(store)
    
    # Run a simple test query to confirm the connection
    # This will raise an error if the server is unreachable
    graph.query("ASK { ?s ?p ?o }")
    
    display(Markdown("✅ **Connection successful!** The database is ready to be queried."))
    
except Exception as e:
    display(Markdown(f"❌ **Connection Failed:** Could not connect to the server.\n\n*Details: {e}*\n\nPlease check the `ENDPOINT_URL` in Step 2 and ensure the server is running."))
    graph = None # Ensure graph is None if connection fails

🚀 Connecting to server at `http://128.178.219.51:7502`...

✅ **Connection successful!** The database is ready to be queried.

### Step 4: Run Example Queries

Now you're ready to go! The `graph` object is your gateway to the database.

Here are a few examples to get you started. You can (and should!) modify these and create new cells to write your own.

In [4]:
# Example 1: Count all triples in the database
# This is a good way to see how much data you're working with.

if graph:
    query_str_count = "SELECT (COUNT(*) AS ?totalTriples) WHERE { ?s ?p ?o }"
    
    display(Markdown("**Running query:** Counting all triples..."))
    
    results = graph.query(query_str_count)
    
    for row in results:
        display(Markdown(f"Total Triples in Database: **{row.totalTriples}**"))
else:
    display(Markdown("⚠️ Skipping query: Database not connected."))

**Running query:** Counting all triples...

Total Triples in Database: **2000002**

In [ ]:
# Example 2: Show 10 triples as a table
# We can use pandas to display the results in a nice table.

if graph:
    query_str_limit = "SELECT * WHERE { ?s ?p ?o } LIMIT 10"
    
    display(Markdown("**Running query:** Getting 10 triples..."))
    
    # Run the query
    results = graph.query(query_str_limit)
    
    # Convert results to a list of dictionaries
    results_list = [row.asdict() for row in results]
    
    # Load into a pandas DataFrame and display
    df = pd.DataFrame(results_list)
    display(df)
    
else:
    display(Markdown("⚠️ Skipping query: Database not connected."))

### Step 5: Your Turn! (Happy Hacking)

This is your canvas. Create new code cells below this one and start building your project.

**Tip:** Don't forget to add `LIMIT 10` to your queries while you are exploring, so you don't accidentally try to print a million rows!

In [ ]:
## Example queries:
# Example 3: Get software repositories and their discipline descriptions from Wikidata (Federated Query)

if graph:
    query_str_limit = """
PREFIX schema: <http://schema.org/> 
PREFIX pulse: <https://open-pulse.epfl.ch/ontology#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    
SELECT ?repoName ?disciplineIRI ?disciplineDescription
WHERE {
  # 1. Get data from our LOCAL graph
  ?repo a schema:SoftwareSourceCode ;
        schema:name ?repoName ;
        pulse:discipline ?disciplineIRI .

  # 2. Connect to an EXTERNAL graph (Wikidata)
  SERVICE <https://query.wikidata.org/sparql> {
    # 3. Ask Wikidata for the description of our ?disciplineIRI
    ?disciplineIRI schema:description ?disciplineDescription .
    
    # Ensure we get the English description
    FILTER(LANG(?disciplineDescription) = "en")
  }
}
LIMIT 100
"""
    
    display(Markdown("**Running query:** Getting 10 triples..."))
    
    # Run the query
    results = graph.query(query_str_limit)
    
    # Convert results to a list of dictionaries
    results_list = [row.asdict() for row in results]
    
    # Load into a pandas DataFrame and display
    df = pd.DataFrame(results_list)
    display(df)
    
else:
    display(Markdown("⚠️ Skipping query: Database not connected."))


**Running query:** Getting 10 triples...

,repoName,disciplineIRI,disciplineDescription
0,Example Software 64899,http://www.wikidata.org/entity/Q83404,"city and county seat in Timiș County, Romania"
1,Example Software 30239,http://www.wikidata.org/entity/Q83404,"city and county seat in Timiș County, Romania"
2,Example Software 54959,http://www.wikidata.org/entity/Q83404,"city and county seat in Timiș County, Romania"
3,Example Software 27819,http://www.wikidata.org/entity/Q83404,"city and county seat in Timiș County, Romania"
4,Example Software 95309,http://www.wikidata.org/entity/Q83404,"city and county seat in Timiș County, Romania"
...,...,...,...
95,Example Software 87999,http://www.wikidata.org/entity/Q83404,"city and county seat in Timiș County, Romania"
96,Example Software 32009,http://www.wikidata.org/entity/Q83404,"city and county seat in Timiș County, Romania"
97,Example Software 42979,http://www.wikidata.org/entity/Q83404,"city and county seat in Timiș County, Romania"
98,Example Software 31979,http://www.wikidata.org/entity/Q83404,"city and county seat in Timiș County, Romania"


In [ ]:
if graph: 
    query_str = """ PREFIX schema: <http://schema.org/> 
    PREFIX pulse: <https://open-pulse.epfl.ch/ontology#> 
    PREFIX wd: <http://www.wikidata.org/entity/>

SELECT ?personName (COUNT(DISTINCT ?repo) AS ?repoCount)
WHERE {
  # 1. Find repositories matching the criteria
  ?repo a schema:SoftwareSourceCode ;
        schema:programmingLanguage ?language ;
        pulse:discipline wd:Q2329 . # wd:Q2329 is the IRI for Chemistry
  
  # 2. Filter for Python (case-insensitive)
#   FILTER(CONTAINS(LCASE(STR(?language)), "python"))
  
  # 3. Find the author of these repos
  ?repo schema:author ?person .
  
  # 4. Ensure the author is a Person and get their name
  ?person a schema:Person ;
          schema:name ?personName .
}
GROUP BY ?person ?personName
ORDER BY DESC(?repoCount)
LIMIT 10
"""

display(Markdown("**Running query:** Finding top 10 authors for Python repos in Chemistry..."))

# Run the query and display in pandas
results = graph.query(query_str)
results_list = [row.asdict() for row in results]
df = pd.DataFrame(results_list)
display(df)

**Running query:** Finding top 10 authors for Python repos in Chemistry...

""
